## 코노스바 텍본 전처리

In [1]:
import json

# JSON 파일 경로 설정
file_path = '코노스바_텍본.json'

# JSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [2]:
len(data)

124

In [3]:
import pandas as pd
import re
pd.set_option('display.max_colwidth', 100000)
df = pd.DataFrame({
    'title': [item['title'] for item in data],
    'content': [item['content'] for item in data]
})

In [4]:
df.content = df.content.replace('\n', '', regex = True)
df.content = df.content.replace('\ufeff', '', regex = True)
df.content = df.content.apply(lambda x: ' '.join(x.split()[1:]).lstrip())

In [5]:
df = df.iloc[3:,:]

In [6]:
# 새로운 데이터프레임을 만들기 위한 리스트
rows = []

# 각 행을 순회하면서 content를 구분자로 분리하고 새로운 행으로 추가
for idx, row in df.iterrows():
    split_content = row['content'].split('◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆◆')
    for content_part in split_content:
        # 각 분리된 내용을 새로운 행으로 추가 (같은 title 유지)
        rows.append({'title': row['title'], 'content': content_part})

# 분리된 행들을 기반으로 새로운 데이터프레임 생성
df = pd.DataFrame(rows)

In [7]:
# 화자: 발언을 추출하기 위한 함수
def extract_dialogues(text):
    dialogues = re.findall(r'\"(.*?)\"', text)  # 따옴표 안의 텍스트를 추출
    speakers = ["카즈마", "아쿠아", "메구밍"]  # 임의로 발언자를 지정 (이 부분은 상황에 맞게 수정)
    
    result = []
    speaker_idx = 0
    
    for dialogue in dialogues:
        # 발언에 대해 순차적으로 화자를 지정 (여기서는 발언이 순서대로라고 가정)
        result.append("\"" + f"{speakers[speaker_idx % len(speakers)]} : {dialogue}" + "\"")
        speaker_idx += 1
        
    return result

In [8]:
df.shape

(257, 2)

In [425]:
# 기본적인 DataFrame 포맷을 설정하는 함수
def format_dataframe(df):
    session = [df['title'].iloc[256]]
    scene = df.content[256]
    dialogue = extract_dialogues(df['content'].iloc[256])

    # 임의의 프로필 및 메구밍의 관계 데이터 설정 (추후 수정 가능)
    profile = {
        "카즈마": {"role": "모험가",
                "abilities": "적 감지, 잠복, 스틸, 드레인터치",
                "personality" : "그럭저럭 평범한 성격, 굳이 나서서 착한 일을 하지도 않고 굳이 피해를 주고 싶어하지도 않는 스타일, 은근히 다정함",
                "characteristic" : "이 멋진 세계에 축복을!의 주인공. 젊은 나이에 어이없는 이유로 사망하여 여신 아쿠아를 만나 이세계로 전이되어 새로운 삶을 시작하는 소년. 그 이전에는 히키니트였다. 이세계로 전생하는 사람에게는 치트급 장비 또는 능력이 하나 주어지는데, 이세계로 가져갈 것을 잘못 골라 처음부터 고생길에 오른다. 직업은 최약체인 모험가이다. 다른 팀원들은 아크 프리스트, 아크 위저드, 크루세이더라는 고급 직업을 가지고 있지만 나머지 팀원들이 어딘가 많이 모자란 모습을 보여 카즈마 자신이 카즈마 파티에서 리더이자 청일점 역할을 하고 있다.",
                "famous_line" : "진정한 남녀평등주의자인 나는 여자를 상대로서도 드롭킥을 날릴 수 있는 남자, 봐줄 거라고는 생각도 하지 마라!"},
        "아쿠아": {"role": "프리스트",
                "abilities": "회복마법, 보조마법, 연회용 장기자랑 스킬",
                "personality" : "늘상 철없고 자존심이 강하며 제멋대로에 허풍만 심한 성격, 심성은 착하지만 주변에 민폐를 많이 끼치는 스타일",
                "characteristic" : "이 멋진 세계에 축복을!의 타이틀 히로인이자, 카즈마와 함께 본작의 또 다른 주인공. 카즈마가 살고 있던 일본에서 죽은 사람들을 안내하는 여신이자 이세계에서 아쿠시즈교단이 섬기는 대상이다.모든 프리스트 계열의 스킬을 사용할 수 있으며, 그 능력 또한 우월하다. 카즈마 파티에서 일단은 서포터 역할을 담당하고 있다. 후배 여신으로 에리스가 있으며 아쿠아는 그녀와 이미 알고 지내던 사이였다.",
                "famous_line" : "이 세상에 존재하는 나의 권속이여.. 물의 여신, 아쿠아가 명하노라『세이크리드 크리에이트 워터』!"},
        "메구밍": {"role": "아크 위자드",
                "abilities": "폭렬마법",
                "personality" : "중2병이 심하고 자존심이 강하며 귀여운 것을 좋아하는 성격, 누구보다 동료를 잘 챙기고 걱정하는 스타일",
                "characteristic" : "이 멋진 세계에 축복을!의 히로인, 스핀오프 이 멋진 세계에 폭염을! 시리즈의 주인공. 마법에 특출난 재능을 가진 홍마족 출신이며, 그중에서도 뛰어난 잠재력을 가지고 있는 유망주. 직업은 아크 위저드지만, 사용할 수 있는 스킬은 오직 폭렬 마법뿐이다. 카즈마 파티에서는 공격을 담당하고 있다. 가족은 아버지 효이자부로, 어머니 유이유이, 여동생 코멧코로 구성되어 있고, 사역마 내지는 반려동물로 춈스케를 데리고 다닌다. 또한, 같은 홍마족인 융융과는 라이벌이자 가장 친한 친구 사이이며, 동급생인 아루에와도 친한 편이다. 홍마의 마을을 떠나 액셀 마을로 여행하는 과정에서 아르칸레티아를 경유하게 되는데, 이때 아쿠시즈교와 인연이 생겨 세실리와 친분이 있는 사이다.",
                "famous_line" : "내 이름은 메구밍! 홍마족 제일의 마법사이자, 폭렬 마법을 펼치는 자!『익스플로-전』!!"},
        "다크니스" : {"role": "크루세이더",
                "abilities": "전사계열 스킬",
                "personality" : "매도당하는 것을 즐기는 마조히스트이며 작품 중 최고의 변태",
                "characteristic" : "이 멋진 세계에 축복을!의 히로인. 카즈마 파티에 마지막으로 가입했으며, 금발벽안에 장신의 금수저 미녀. 독실한 에리스 교도이며, 타인을 위해 어떠한 고통도 감수하며 희생정신 또한 출중하여 귀감이 되는 크루세이더. 크루세이더 중에서도 극도로 강한 방어력과 상태이상 저항 능력을 가지고 있으며, 카즈마 파티에서 탱킹 역할을 맡고 있다. 아버지의 이름은 이그니스이며 어머니는 오래전에 이미 사망하여 작중에 등장하지 않는다. 크리스와는 모험가 생활을 시작하면서 부터 친하게 지내던 사이이다. 여담이지만 어떤 뚱보 인간 쓰레기 귀족과의 결혼식은 식 당일 무산됐지만 귀족들간의 결혼은 혼인신고를 한 뒤 식을 올리는 거라 법적으로는 이미 기혼 상태였고, 지금은 이혼녀 상태다. 때문에 그걸로 놀림당한 적이 있다. 작품 팬들도 그녀를 바츠이치라고 놀리는 밈이 있다.",
                "famous_line" : "어이, 카즈마. 내가 이렇게 보여도 소녀 나부랭이란 말이다. (울먹이며) 내 몸은 그렇게까지 단단하지 않아!!"}
    }

    return session, scene, dialogue, profile

In [426]:
# 결과 데이터프레임과 딕셔너리 구성
session, scene, dialogue, profile, = format_dataframe(df)

In [427]:
speakers = ['카즈마', '아쿠아', '메구밍', '다크니스']

In [428]:
# JSON 파일로 저장하는 함수
def save_list_to_json(file_name, data):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=0)

# 사용 예시
save_list_to_json('dialogue_output.json', dialogue)

In [418]:
# JSON 파일에서 데이터를 불러오는 함수
def load_json_to_list(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 사용 예시
dialogue = load_json_to_list('dialogue_output.json')

In [419]:
data = {
    'Session_195' :{
        'position' : session,
        'speakers' : speakers,
        'scene' : scene,
        'dialogue' : dialogue,
        'profile' : profile,
    }
}

In [420]:
data

{'Session_195': {'position': ['코노스바! - 에필로그'],
  'speakers': ['카즈마', '아쿠아', '메구밍', '다크니스'],
  'scene': '평소와 똑같은 평소의 방. 지루할뿐더러 아무런 변화도 없는 나의 직장에, 이 방의 단골손님이 나타났다. "여어." 그는 가볍게 손을 흔들며 인사를 건넸다. 여어, 같은 말을 할 상황이 아니지만……. "카즈마 씨……. 정말이지, 어째서 당신은……" 어째서 이 사람은 이렇게 간단히 죽어 버리는 걸까. 마왕을 쓰러뜨린 사람이 아무렇지도 않게 이곳에 오는 건 그만둬줬으면 한다. 그보다도, 이런 곳에서 늘어지는 것도 그만둬줬으면 한다. 양손을 뒤통수에 돌린 채 아무렇게나 바닥에 앉아 만족스럽다는 듯이 늘어지고 있다. ――이곳은 이 세계의 사자(死者)들의 영혼이 불려오는 장소. 결코 늘어질 만한 장소가 아닌데……. "이야~, 역시 여기가 가장 편하다니까요. 뭐랄까, 에리스 님을 보는 것만으로도 치유된단 말이죠." 그런 말을 하면서, 그는 팔을 베개로 삼아 드러누웠다. "……정말이지. 그 후부터 정말 힘들었다구요? 제가 여기에 돌아오기 위해서 대체 얼마나 고생을 했는지……" "아니 그래도 말이죠, 에리스 님도 참, 마을에 돌아왔더니 이미 없어지셨잖아요. 제대로 바래다 드리려고 했었는데." ……바래다줘? 아니, 그런 것보다도. "그, 에리스 님이라는 호칭은 그만두시지 않겠나요? 저의 정체도 알고 계시고, 그냥 불러도 괜찮다구요? 그 말투도, 조금 더 가벼운 느낌으로……" "그럼 에리스 님도 평소 같은 말투로 말한다면 평범하게 부를게요." "……이 모습일 때는 아무래도 이런 말투가……. 그런데, 어쩐지 피곤해 보이시네요? 이번엔 대체 무슨 일인가요? 완전히 평화로워졌길래 그다지 하계를 관찰하지 않았는데요……" 대체 어떤 몬스터한테 살해당한 걸까. 이젠 그에게 쓴맛을 보게 할 상대 같은 건 그리 많지 않을 텐데. "뭐, 이것저것 있어서요. 것보다, 조금 우쭐거렸을 뿐인데 저택에서 쫓겨났거든요. 

In [421]:
# JSON 파일로 저장하는 함수
def save_list_to_json(file_name, data):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 사용 예시
save_list_to_json('Json_Data/Session_195.json', data)

## 이멋폭 전처리

In [1]:
import json
import re

# JSON 파일 경로 설정
file_path = 'Preprocessing/이멋진폭염_텍본.json'

# JSON 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [2]:
data[0]['content']

'이전 화 목록 다음 화\n속·이 멋진 세계에 폭염을!\n프롤로그\nhttp://sneakerbunko.jp/novel_data/123468540706/_SWF_Window.html\n\n\n이것은 그 사람과 함께 갔던 불꽃놀이 대회의 밤의 이야기이다.\n내가 그 사람과의 약속을 지키지 못해서, 조금이지만 침울해져 뚜벅뚜벅 걸어가고 있을 때 였다.\n언다인이란 이름의 평판 나쁜 귀족의 저택.\n그곳에, 딱 보기에도 수상한 사람들이 방 안 쪽을 보고 있었다.\n기억에 있는 그 얼굴에 가슴이 두근거린다.\n뭐야, 난 이렇게나 쉬운 여자였던 건가.\n좀 더 행실이 좋은 사람인줄 알았는데......\n나는, 결심하고, 두 사람에게 말을 건다.\n\n"저, 저기...... 혹시, 거기에 계신 건 은발 도적단의 두분이 아니신가요?"\n\n등 뒤에서 말이 걸려오자 \'깜짝\'하고 몸을 떠는 그 사람에게, 눈치채보니 나는 흥분한 목소리로 자기소개하고 있었다.\n입술을 부들부들거리는 그 둘에게, 나는 계속 신경 쓰였던 것을 묻는다.\n\n"두분이 왕성에 숨어 드신것은, 왕녀님이 위험한 신기를 가지고 계시기 때문에, 그 분을 지키려고 하신 겁니까!?"\n그런 나의 질문에.\n"아아 그래. 우리들은 세상에서 말하는 귀족. 평소에는 서민의 말투지만, 그것이 아무리 왕녀라도, 여린 소녀가 위험에 노출되는 걸 무시할 수는 없지. 곤란한 사람이 있다면, 그곳에 귀족의 저택이든, 왕성이든 어디든지 간에 숨어들어. 그것이 가면도적단이다"\n\n자세한 내용은 다를지도 모르지만, 그 사람은 확실히 그리 말했다.\n\n"메구밍 이라고 말했었지? 사실은 우리들은, 이 저택에 잠들어 있는 어느 물건을 노리고 있어. 그것은 인류의 미래를 위해서 필요한 것. 도둑질이라는 행위는 확실히 칭찬받을 만한 일은 아니지. 하지만 그것은 우리들에게 있어서, 우리들의 목에 상금이 걸렸다 해도, 하지 않으면 안되는 일이야."\n\n굳은 결심은 안에 담은, 어째선지 보기만 해도 기시감이 드는 가면 아래의 그 눈동자.\

In [3]:
# "" 사이의 텍스트를 추출하는 함수
def extract_text_in_quotes(data):
    result = []
    pattern = r'"(.*?)"'  # 정규식: "" 사이의 텍스트 추출

    for item in data:
        # 정규식으로 "" 사이의 텍스트 추출
        match = re.findall(pattern, item['content'])
        if match:
            result.extend(match)
        else:
            result.append([])  # ""가 없는 경우 빈 리스트

    return result

# 결과 확인
extracted_text = extract_text_in_quotes(data)

In [4]:
data = {
    'dialogue' : extracted_text
}

In [5]:
data

{'dialogue': ['저, 저기...... 혹시, 거기에 계신 건 은발 도적단의 두분이 아니신가요?',
  '두분이 왕성에 숨어 드신것은, 왕녀님이 위험한 신기를 가지고 계시기 때문에, 그 분을 지키려고 하신 겁니까!?',
  '아아 그래. 우리들은 세상에서 말하는 귀족. 평소에는 서민의 말투지만, 그것이 아무리 왕녀라도, 여린 소녀가 위험에 노출되는 걸 무시할 수는 없지. 곤란한 사람이 있다면, 그곳에 귀족의 저택이든, 왕성이든 어디든지 간에 숨어들어. 그것이 가면도적단이다',
  '메구밍 이라고 말했었지? 사실은 우리들은, 이 저택에 잠들어 있는 어느 물건을 노리고 있어. 그것은 인류의 미래를 위해서 필요한 것. 도둑질이라는 행위는 확실히 칭찬받을 만한 일은 아니지. 하지만 그것은 우리들에게 있어서, 우리들의 목에 상금이 걸렸다 해도, 하지 않으면 안되는 일이야.',
  '우리들은 이제부터 저택에 숨어들거야. 그리고, 마왕군에 대항할 카드 중 하나를 손에 넣을거야. 네가 통보한다고 한다면 막을 순 없지만......, 하지만 믿어줬으면 해, 이건 인류를 위해서야',
  '멈췃..... 멈춰! 뭐하시는 거에요!',
  "'뭐하시는 거에요'가 아니라구요! 이건 모험가가 파티멤버를 모집하기 위한 게시판입니다. 놀이 상대를 찾으신다면 다른 곳을 찾아봐 주세요!",
  '놀이 상대라니 실례네요! 이건 엄연히 동료모집이에요. 제가 벽보를 붙힌 장소에 불만이 있으면 한번 말해보시죠!',
  '놀이가 아니라면 죄송하지만요! 불만이 있는 건 붙인 장소가 아니라, 모집항목의 부분입니다!',
  "'도적직 구함. 정의를 위해서라면 범죄행위도 불사하지 않는, 열정 있으신 분 한정. 주요 임무 내용은 귀족령에 강습 같은 것들......'",
  '.....어쩔 수 없군요. 본래라면 도적직에 한정하고 싶었지만, 다른 직업이라도 괜찮아요. 고치도록 하죠',
  '그곳이 아니라고요 문제는! 길드의 게시판을 사용해서 범죄자 동료를 모집하지 말아 달라고 하고 있는 거에요!',
  "그

In [6]:
# JSON 파일로 저장하는 함수
def save_list_to_json(file_name, data):
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

# 사용 예시
save_list_to_json('flame_dialogue.json', extracted_text)